In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from pprint import pprint

In [2]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

model = keras.models.load_model(
    'ucf11_weights/10-0.87.hdf5',
    custom_objects={'KerasLayer':hub.KerasLayer} 
)

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              5919312   
_________________________________________________________________
dense (Dense)                (None, 11)                14091     
Total params: 5,933,403
Trainable params: 5,872,795
Non-trainable params: 60,608
_________________________________________________________________


In [4]:
feature_extractor = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)),
        model.layers[0]
    ], name='feature_extractor'
)

feature_extractor.summary()

Model: "feature_extractor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              5919312   
Total params: 5,919,312
Trainable params: 5,858,704
Non-trainable params: 60,608
_________________________________________________________________


In [5]:
os.mkdir('UCF11_updated_npy')
os.mkdir('UCF11_updated_npy/train')
os.mkdir('UCF11_updated_npy/valid')

In [12]:
max_frames = 10
SAVE_DIR = 'UCF11_updated_npy/'
train_df = pd.read_csv('ucf11_train_vid.csv')
valid_df = pd.read_csv('ucf11_valid_vid.csv')

In [13]:
for i, elem in tqdm(train_df.iterrows(),
                    total=len(train_df)):
    label = elem['label']
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_ = frame.copy()
        frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        frame_ = cv2.resize(frame_, (256, 256))
        frame_ = frame_.astype('float32')
        frame_ = frame_ / 255.
        
        frames.append(frame_)
        if len(frames) == max_frames:
            break
    cap.release()
    
    frames = np.array(frames)
    features = feature_extractor.predict(frames)
    
    file_name = SAVE_DIR + f'train/{label}_{i}.npy'
    np.save(file_name, features)

100%|██████████| 220/220 [00:21<00:00, 10.22it/s]


In [14]:
print(len(glob(SAVE_DIR + 'train/*')))

220


In [15]:
for i, elem in tqdm(valid_df.iterrows(),
                    total=len(valid_df)):
    label = elem['label']
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_ = frame.copy()
        frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        frame_ = cv2.resize(frame_, (256, 256))
        frame_ = frame_.astype('float32')
        frame_ = frame_ / 255.
        
        frames.append(frame_)
        if len(frames) == max_frames:
            break
    cap.release()
    
    frames = np.array(frames)
    features = feature_extractor.predict(frames)
    
    file_name = SAVE_DIR + f'valid/{label}_{i}.npy'
    np.save(file_name, features)

100%|██████████| 55/55 [00:05<00:00, 10.12it/s]


In [16]:
print(len(glob(SAVE_DIR + 'valid/*')))

55
